In [2]:
import pandas as pd
import numpy as np

In [25]:
Kreis_id=[1,2,3,4,5,6,7,8,9,10,11,12]

## Area in km^2
Kreis_area=[1.80, 11.07, 8.65, 2.90, 1.99, 5.10, 15.02, 4.81, 12.07, 9.09, 13.42, 5.97]

Kreis_pop_2021=[5817, 36025, 50104, 29023, 15804, 35355, 38663, 17818, 57077, 41017, 76975, 32654]

## Density in population/km^2
Kreis_density=[3232, 3254, 5792, 10008, 7942, 6932, 2574, 3704, 4729, 4512, 5736, 5470]

Kreis_info=pd.DataFrame(list(zip(Kreis_id, Kreis_area, Kreis_pop_2021,Kreis_density)),
               columns =['Kreis_id', 'Kreis Area (km^2)', 'Kreis Population', 'Kreis Density (population/km^2)'])

In [26]:
Kreis_info

,Kreis_id,Kreis Area (km^2),Kreis Population,Kreis Density (population/km^2)
0,1,1.80,5817,3232
1,2,11.07,36025,3254
2,3,8.65,50104,5792
3,4,2.90,29023,10008
4,5,1.99,15804,7942
5,6,5.10,35355,6932
6,7,15.02,38663,2574
7,8,4.81,17818,3704
8,9,12.07,57077,4729
9,10,9.09,41017,4512


In [27]:
Kreis_info.to_csv("Krei_info", sep=',', index=False, encoding='utf-8')

### 
Four KPIs:

a_1: Number of Migros stores in a Kreis/Total number of supermakts (the lower, the chance of the kreis higher)

a_2: Current population density in the Kreis / population density of Zurich (the higher, the chance of the kreis higher)

a_3: Average income in the Kreis / Current average income in Zurich (the higher, the chance of the kreis higher)

a_4: Future prediction of population density change / population density of Zurich (the higher, the chance of the kreis higher)

To open a new store in a Kreis, we can propose a linear combination model with weights for each KPI. (An average shopper gets groceries 1.6 times per week)

Relevance_factor = a_1*w_1 + a_2*w_2 + a_3*w_3 + a_4*w_4

We compute this relevance factor for each kreis and see which one gets a higher relevance factor.

In [152]:
Aldi_store_per_kreis=[2, 2, 0, 0, 1, 0, 1, 0, 2, 0, 3, 1]
Lidl_store_per_kreis=[1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 3, 1]
Coop_store_per_kreis=[6, 5, 6, 4, 5, 3, 3, 4, 9, 6, 8, 3]
Migros_store_per_kreis=[4, 4, 3, 3, 3, 2, 3, 2, 3, 2, 3, 1]

Total_store_num=[x_1 + x_2 + x_3 + x_4 for x_1, x_2, x_3, x_4 in zip(Aldi_store_per_kreis, Lidl_store_per_kreis,Coop_store_per_kreis,Migros_store_per_kreis)]

In [265]:
# KPI 1: a_1: number of migros stores/total number of stores in a kreis

a_1=[y_1/ y_2 for y_1, y_2 in zip(Migros_store_per_kreis,Total_store_num)]

In [266]:
# KPI 2: a_2: Current population denisty

a_2=[0.675171,0.694047,1.212819,2.122174,1.655881,1.458411,0.542626,0.768098,1.005600,0.947106,1.205974,1.144783]

In [267]:
# KPI 3: a_3: Avergae income in a kreis (KCHF)
den_1 = pd.read_csv("../data/processed/average_income_per_kreis_sorted.csv")

mean_income=den_1['Median value of taxable income'].mean()
income_per_kreis=[76.38, 68.53, 58.32, 52.2, 71.07, 74.13, 89.27, 88.97, 57.0, 67.68, 55.3, 44.67]

a_3 = [i /mean_income for i in income_per_kreis]

In [268]:
# KPI 4: a_4: future population denisty 2040

future_pop_density=[0.589022, 0.693816,1.206202,1.966494,1.746213,1.396008,0.527892,0.728034,1.000356,0.897879,1.262164,1.307432]

a_4 = [z_1- z_2 for z_1, z_2 in zip(future_pop_density,a_2)]
a_4_percentage= [i *100 for i in a_4]

In [272]:
w_1 = input('Select a weight for 1st KPI?\n')
w_2 = input('Select a weight for 2nd KPI?\n')
w_3 = input('Select a weight for 3rd KPI?\n')
w_4 = input('Select a weight for 4th KPI?\n')
weight_fac=[float(w_1),float(w_2),float(w_3),float(w_4)]

Rel_fac_a_1=[i * weight_fac[0] for i in a_1]
Rel_fac_a_2=[i * weight_fac[1] for i in a_2]
Rel_fac_a_3=[i * weight_fac[2] for i in a_3]
Rel_fac_a_4=[i * weight_fac[3] for i in a_4]


Total_rel_fac=[x_1 + x_2 + x_3 + x_4 for x_1, x_2, x_3, x_4 in zip(Rel_fac_a_1,Rel_fac_a_2,Rel_fac_a_3,Rel_fac_a_4)]


Kreis_recommendation=pd.DataFrame(list(zip(Kreis_id, Total_rel_fac, a_1, a_2, a_3, a_4_percentage)), 
               columns =['Kreis #', 'Relevance Factor', 'Migros Market Share', 'Population Density', 'Normalized AVG Income','Population Change by 2040 %'])


Select a weight for 1st KPI?
 2
Select a weight for 2nd KPI?
 .5
Select a weight for 3rd KPI?
 1
Select a weight for 4th KPI?
 3


In [273]:
Kreis_recommendation.set_index(['Kreis #'])

,Relevance Factor,Migros Market Share,Population Density,Normalized AVG Income,Population Change by 2040 %
Kreis #,,,,,
1,1.835204,0.307692,0.675171,1.140681,-8.6149
2,2.097050,0.363636,0.694047,1.023447,-0.0231
3,2.057526,0.300000,1.212819,0.870968,-0.6617
4,2.230760,0.428571,2.122174,0.779570,-15.5680
5,2.760316,0.300000,1.655881,1.061380,9.0332
6,2.449075,0.400000,1.458411,1.107079,-6.2403
7,2.417438,0.428571,0.542626,1.333184,-1.4734
8,2.259227,0.333333,0.768098,1.328704,-4.0064
9,1.766894,0.214286,1.005600,0.851254,-0.5244


In [274]:
max_factor=Kreis_recommendation['Relevance Factor'].max()
df2=Kreis_recommendation.loc[Kreis_recommendation['Relevance Factor'] == max_factor]
recom_kreis=df2.values.tolist()[0][0]

print(f"Based on the given weights, the recommendation is to consider openning a new Migros store in Kreis {int(recom_kreis)}.")

Based on the given weights, the recommendation is to consider openning a new Migros store in Kreis 5.
